# Association Analysis Project

### What is association analysis?

Association analysis is an analysis that discovers the probability of different sets of items appearing in the same record. These relationships between items are expressed as __Association Rules__. An association rule might look like this: If items A and B are in a transaction or record, item C is likely to be there as well. These rules are often denoted as {antecedent} -> {consequent}. We will see more examples of this in the demonstration below.

<br>Association analysis, often referred to as __Market Basket Analyis__, is a valuable data mining technique. It can be especially valuable in settings like retail or food & beverage, where a business might want to know how different products relate to one another. 

<br>For example, Association Analysis can tell us how frequently __Product A__ and __Product B__ were purchased together. This information is then compared to the entire dataset to understand whether __Product A__ and __Product B's__ relationship is a meaningful opportunity for cross-selling, promotion, or another lever to drive sales. This exercise is then extended to all possible __itemsets__.

### Project Goal

In learning about association analysis, I found that most resources typically took input as a list of products and records: (A, B, C),(B,C),(A), etc. In my experience this is not a common format for transaction data.

Another limitation of existing demonstrations is that they typically present small datasets, no more than a few hundred rows. Again, in my experience, datasets are rarely that small in business. 

The goal of this project is to model an association analysis using large datasets (100,000+ records) that are formatted with columns as items or products, and with rows as records in the dataset.

This project is written in python, using popular data science libraries like pandas, numpy, and mlxtend. 

### Project Outline

1. Create dummy dataset(s) for association analysis
1. Read the dataset(s) into python data structure (pandas dataframe)
1. Convert the data into the appropriate format for association analysis
1. Exploratory Analysis
1. Get Frequent Itemsets
1. Run Association Rules algorithm on Frequent Itemsets

In [1]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth, association_rules

In [2]:
def read_files_into_df_list(file_paths):
    """
    Read list of file paths into dataframes
    Return list of dataframes
    Notify user of successful read
    """
    
    df_list = []
    
    # check if any file paths provided
    # if yes, read first file into df
    if len(file_paths) > 0:
        
        df = pd.read_csv(file_paths[0],index_col=0)
        df_list.append(df)
        
        # if more than 1 file, iterate through remaining files
        # append df to df_list
        if len(file_paths) > 1:
            for file_path in file_paths[1:]:
                df = pd.read_csv(file_path,index_col=0)
                df_list.append(df)
            print("Finished reading " + str(len(df_list)) + " files to dataframe.")
            return df_list
        else:
            print("Finished reading " + str(len(df_list)) + " file to dataframe.")
            return df_list
    else:
        print("You did not input a file. Please try again.")

def bool_encoding_df(df_list):
    """
    Iterate through df_list
    Replace all integers > 1 with 1
    Replace all NaN with 0
    """
    
    for df in df_list:
        col_list = df.columns
        for col in col_list:
            # replace int > 1 with 1
            df.loc[df[col]>1,col] = 1
            # replace NaN with 0
            df[col].fillna(0,inplace=True)
            # convert to bool
            df[col] = df[col].astype('bool',copy=False)
    return df_list

### Create Dummy Dataset(s) for Association Analysis

In [3]:
# set parameters for dummy dataset
nums = 10
proba_nums = [0.9,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01]
product_list = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
size = (50000,len(product_list))

In [ ]:
# create df with pseudo-random numbers, given probability
df = pd.DataFrame(np.random.choice(nums,size,replace=True,p=proba_nums))

In [ ]:
# replace int with letters for columns
df.columns = product_list

# drop all rows with only 0s
df = df[(df!=0).any(1)]

In [ ]:
# write df to csv
df.to_csv('Association Analysis Project - DataX.csv')

### Read Dataset(s) Into Dataframe

In [4]:
file_paths = ['Association Analysis Project - Data1.csv','Association Analysis Project - Data2.csv',
              'Association Analysis Project - Data3.csv','Association Analysis Project - Data4.csv']

In [5]:
df_list = read_files_into_df_list(file_paths)

Finished reading 4 files to dataframe.


In [6]:
df_list

[       A  B  C  D  E  F  G  H  I  J  ...  Q  R  S  T  U  V  W  X  Y  Z
 0      0  0  8  0  8  0  0  0  0  0  ...  2  0  0  0  0  0  0  0  0  0
 1      0  0  5  0  0  0  0  0  0  0  ...  0  0  1  0  0  0  0  0  0  0
 2      0  0  0  0  6  8  0  0  0  7  ...  0  0  0  0  0  0  1  0  0  0
 3      0  0  0  5  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  5  1  0
 4      0  8  3  0  0  0  0  0  0  0  ...  0  0  0  7  0  8  0  0  0  0
 ...   .. .. .. .. .. .. .. .. .. ..  ... .. .. .. .. .. .. .. .. .. ..
 49995  0  0  0  0  0  0  0  0  1  0  ...  0  0  0  0  0  0  0  0  6  0
 49996  0  0  0  2  0  0  0  0  0  0  ...  0  0  0  0  0  0  5  0  0  0
 49997  0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  6  0  0  0  0  0
 49998  0  0  0  0  0  0  0  0  0  1  ...  0  0  0  8  0  0  0  0  0  0
 49999  0  0  0  0  0  2  0  0  0  0  ...  0  0  0  0  0  0  0  0  0  0
 
 [46761 rows x 26 columns],
        A  B  C  D  E  F  G  H  I  J  ...  Q  R  S  T  U  V  W  X  Y  Z
 0      0  0  0  0  0  0  0  0  0 

### Convert Dataframes to Boolean Format, Concatenate Dataframes

In [7]:
df_bool_list = bool_encoding_df(df_list)

In [8]:
df_bool_list

[           A      B      C      D      E      F      G      H      I      J  \
 0      False  False   True  False   True  False  False  False  False  False   
 1      False  False   True  False  False  False  False  False  False  False   
 2      False  False  False  False   True   True  False  False  False   True   
 3      False  False  False   True  False  False  False  False  False  False   
 4      False   True   True  False  False  False  False  False  False  False   
 ...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
 49995  False  False  False  False  False  False  False  False   True  False   
 49996  False  False  False   True  False  False  False  False  False  False   
 49997  False  False  False  False  False  False  False  False  False  False   
 49998  False  False  False  False  False  False  False  False  False   True   
 49999  False  False  False  False  False   True  False  False  False  False   
 
        ...      Q      R      S      

In [9]:
# concat the list of dataframes into one
df = pd.concat(df_list,axis=0,ignore_index=True)

### Exploratory Analysis of Dataframe

In [10]:
df

,A,B,C,D,E,F,G,H,I,J,...,Q,R,S,T,U,V,W,X,Y,Z
0,False,False,True,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
1,False,False,True,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,False,False,False,False,True,True,False,False,False,True,...,False,False,False,False,False,False,True,False,False,False
3,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
4,False,True,True,False,False,False,False,False,False,False,...,False,False,False,True,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186975,False,True,False,True,False,False,False,False,False,True,...,True,False,False,False,False,False,False,False,False,False
186976,False,False,False,False,True,False,True,True,False,False,...,False,False,False,False,False,False,False,False,True,False
186977,False,False,False,False,False,False,True,False,True,False,...,False,False,False,False,False,False,False,True,False,True
186978,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186980 entries, 0 to 186979
Data columns (total 26 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   A       186980 non-null  bool 
 1   B       186980 non-null  bool 
 2   C       186980 non-null  bool 
 3   D       186980 non-null  bool 
 4   E       186980 non-null  bool 
 5   F       186980 non-null  bool 
 6   G       186980 non-null  bool 
 7   H       186980 non-null  bool 
 8   I       186980 non-null  bool 
 9   J       186980 non-null  bool 
 10  K       186980 non-null  bool 
 11  L       186980 non-null  bool 
 12  M       186980 non-null  bool 
 13  N       186980 non-null  bool 
 14  O       186980 non-null  bool 
 15  P       186980 non-null  bool 
 16  Q       186980 non-null  bool 
 17  R       186980 non-null  bool 
 18  S       186980 non-null  bool 
 19  T       186980 non-null  bool 
 20  U       186980 non-null  bool 
 21  V       186980 non-null  bool 
 22  W       186980 non-n

In [12]:
df.describe()

,A,B,C,D,E,F,G,H,I,J,...,Q,R,S,T,U,V,W,X,Y,Z
count,186980,186980,186980,186980,186980,186980,186980,186980,186980,186980,...,186980,186980,186980,186980,186980,186980,186980,186980,186980,186980
unique,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
top,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
freq,167158,167027,166961,166917,167048,167168,166961,166982,166895,166873,...,167184,166848,166948,167237,166878,167316,167107,167098,166973,167070


### Get Frequent Itemsets Using FPGrowth Algorithm

In [13]:
frequent_itemsets = fpgrowth(df, min_support=0.001, use_colnames=True)

In [14]:
frequent_itemsets

,support,itemsets
0,0.107065,(C)
1,0.106600,(E)
2,0.105872,(Q)
3,0.107134,(S)
4,0.107536,(J)
...,...,...
2407,0.001059,"(I, S, M)"
2408,0.001027,"(M, I, K)"
2409,0.001161,"(I, D, M)"
2410,0.001064,"(D, S, M)"


### Get Association Rules from Frequent Itemsets

In [15]:
association_rule_list = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.1)

In [16]:
association_rule_list.sort_values('confidence',ascending=False,inplace=True)

In [17]:
association_rule_list

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
868,"(W, P)",(K),0.010472,0.106530,0.001369,0.130746,1.227312,0.000254,1.027858
867,"(K, P)",(W),0.010916,0.106284,0.001369,0.125429,1.180127,0.000209,1.021890
866,"(K, W)",(P),0.010932,0.107049,0.001369,0.125245,1.169976,0.000199,1.020801
1558,"(B, X)",(E),0.010691,0.106600,0.001321,0.123562,1.159120,0.000181,1.019353
1557,"(E, X)",(B),0.010696,0.106712,0.001321,0.123500,1.157321,0.000180,1.019154
...,...,...,...,...,...,...,...,...,...
3372,"(K, D)",(A),0.010803,0.106011,0.001080,0.100000,0.943295,-0.000065,0.993321
973,"(F, M)",(E),0.010643,0.106600,0.001064,0.100000,0.938090,-0.000070,0.992667
1989,"(I, T)",(G),0.010803,0.107065,0.001080,0.100000,0.934013,-0.000076,0.992150
3803,"(Y, H)",(U),0.010643,0.107509,0.001064,0.100000,0.930156,-0.000080,0.991657


Check out more of my data analysis projects and work at:

__Website__ https://benjaminwyndham.com/

__LinkedIn__ https://www.linkedin.com/in/benwyndham/

__GitHub__ https://github.com/bwyndham